# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.104/12124/1 Dashboard: http://10.20.0.104:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,967,Victor,0.3945672821537938,0.5419757507780067
2000-01-01 00:00:01,1053,Oliver,-0.6388550551770606,-0.6632464716960764
2000-01-01 00:00:02,948,Oliver,-0.6922846396040636,0.6406393166734488
2000-01-01 00:00:03,1068,Charlie,-0.2722330370947441,-0.4883022143141711
2000-01-01 00:00:04,1008,Quinn,-0.5726129194726051,0.42751589153936176
2000-01-01 00:00:05,967,Frank,0.9746149083075057,0.8170374666285298
2000-01-01 00:00:06,1030,Edith,-0.949187613998338,0.560588639205889
2000-01-01 00:00:07,984,Michael,0.6029551552682377,-0.32857055891140385
2000-01-01 00:00:08,988,Frank,-0.44776822665328386,0.21721790470428992


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1038,Tim,-0.07899335252925943,-0.9902211692766023
2000-01-30 00:00:01,1003,Bob,-0.6539619695404455,-0.08478188778235496
2000-01-30 00:00:02,1025,Sarah,-0.7515379522279046,-0.16441610306939403
2000-01-30 00:00:03,1030,Victor,0.887838778430386,0.3759100881596258
2000-01-30 00:00:04,1002,Zelda,-0.5245095411521565,0.21901395907654742
2000-01-30 00:00:05,971,Wendy,0.29079305227416286,-0.06583575010247467
2000-01-30 00:00:06,1029,Tim,-0.6278863375574744,-0.2669611640317411
2000-01-30 00:00:07,1005,Zelda,0.09676030607651165,-0.14690164938858596
2000-01-30 00:00:08,981,Bob,-0.24542456611171337,0.5772124828040632


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,967,Victor,0.394567,0.541976
1,2000-01-01 00:00:01,1053,Oliver,-0.638855,-0.663246
2,2000-01-01 00:00:02,948,Oliver,-0.692285,0.640639
3,2000-01-01 00:00:03,1068,Charlie,-0.272233,-0.488302
4,2000-01-01 00:00:04,1008,Quinn,-0.572613,0.427516


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,967,Victor,0.394567,0.541976
1,2000-01-01 00:00:01,1053,Oliver,-0.638855,-0.663246
2,2000-01-01 00:00:02,948,Oliver,-0.692285,0.640639
3,2000-01-01 00:00:03,1068,Charlie,-0.272233,-0.488302
4,2000-01-01 00:00:04,1008,Quinn,-0.572613,0.427516


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.63 s, sys: 596 ms, total: 5.23 s
Wall time: 3.42 s


name
Alice      -0.002649
Bob         0.001272
Charlie    -0.000301
Dan         0.001170
Edith       0.001667
Frank      -0.000985
George      0.000672
Hannah      0.001836
Ingrid      0.000011
Jerry       0.001143
Kevin      -0.002294
Laura       0.002225
Michael    -0.000101
Norbert    -0.002274
Oliver      0.000170
Patricia   -0.002258
Quinn      -0.000271
Ray        -0.000726
Sarah       0.000699
Tim         0.001875
Ursula     -0.001184
Victor     -0.001534
Wendy       0.001203
Xavier      0.001558
Yvonne      0.000056
Zelda      -0.001405
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.46 s, sys: 152 ms, total: 1.61 s
Wall time: 1.08 s


name
Alice      -0.002649
Bob         0.001272
Charlie    -0.000301
Dan         0.001170
Edith       0.001667
Frank      -0.000985
George      0.000672
Hannah      0.001836
Ingrid      0.000011
Jerry       0.001143
Kevin      -0.002294
Laura       0.002225
Michael    -0.000101
Norbert    -0.002274
Oliver      0.000170
Patricia   -0.002258
Quinn      -0.000271
Ray        -0.000726
Sarah       0.000699
Tim         0.001875
Ursula     -0.001184
Victor     -0.001534
Wendy       0.001203
Xavier      0.001558
Yvonne      0.000056
Zelda      -0.001405
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.3 s, sys: 74.8 ms, total: 1.38 s
Wall time: 974 ms


name
Alice      -0.002649
Bob         0.001272
Charlie    -0.000301
Dan         0.001170
Edith       0.001667
Frank      -0.000985
George      0.000672
Hannah      0.001836
Ingrid      0.000011
Jerry       0.001143
Kevin      -0.002294
Laura       0.002225
Michael    -0.000101
Norbert    -0.002274
Oliver      0.000170
Patricia   -0.002258
Quinn      -0.000271
Ray        -0.000726
Sarah       0.000699
Tim         0.001875
Ursula     -0.001184
Victor     -0.001534
Wendy       0.001203
Xavier      0.001558
Yvonne      0.000056
Zelda      -0.001405
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html